# Lab 6-1: Softmax Classification

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Softmax(목적 함수)

$$ P(class=i) = \frac{e^i}{\sum e^i} $$

In [3]:
# 예시 데이터
z = torch.FloatTensor([1, 2, 3])

# softmax 적용
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


Since they are probabilities, they should add up to 1. Let's do a sanity check.

In [4]:
# 확률 값이므로 당연히 총합 1
hypothesis.sum()

tensor(1.)

## Cross Entropy Loss (Low-level) - cost 함수

$$ L = \frac{1}{N} \sum - y \log(\hat{y}) $$

In [5]:
z = torch.rand(3, 5, requires_grad=True) # 0~1 사이의 값을 (3,5)사이즈로 균등하게 생성
hypothesis = F.softmax(z, dim=1) # 소프트 맥스 함수를 열 단위로(x축) 적용 -> 샘플 3개 클래스가 5개가 되는 확률 분포가 된다.
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [6]:
# 정답 레이블을 임의로 만듦

y = torch.randint(5, (3,)).long() # 정수
print(y)

tensor([0, 2, 1])


In [7]:
# 원핫 인코더를 해줘야함

y_one_hot = torch.zeros_like(hypothesis) # 빈 껍데기 0인 유니폼 분포
y_one_hot.scatter_(1, y.unsqueeze(1), 1) # dim=1 행을 기준으로 y의 2,3,2 레이블을 1로 인코더한다

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [8]:
# -sum(정답 레이블 - log(예측값))을 평균내어 cost 함수 만듦

cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


## `torch.nn.functional`으로 cross entropy 정의

In [9]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [10]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

### 애초에 log softmax를 제공함

In [11]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [12]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward>)

In [13]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

### nll(negative log likelihood)도 제공하지만, 그냥 크로스 엔트로피 함수를 이용하면 됨
________________________________________________________________________________________________

# <center> 모델 학습하기 <center/>

## Training with Low-level Cross Entropy Loss

In [14]:
# 데이터 정의

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]

y_train = [2, 2, 2, 1, 1, 1, 0, 0]   # 클래스 정의
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [15]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)  # w의 차원은 (변수 개수, class 개수)로 정의
b = torch.zeros(1, requires_grad=True)       # 이거는 이해안가는데 그냥 1개로 고정한 것 같음(?)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # 합성곱을 위해 matmul 이용
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


##  `F.cross_entropy`으로 학습 시키기

In [16]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train) # 원핫 인코딩을 해주기떄문에 scatter가 필요없음

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


## `nn.Module`로 학습하기

In [17]:
# 소프트맥스 함수 클래스 만들어주기
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # (입력 변수, 클래스)

    def forward(self, x):
        return self.linear(x)

In [18]:
model = SoftmaxClassifierModel()

In [19]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train) # 모델 클래스에 x_train 삽입

    # cost 계산
    cost = F.cross_entropy(prediction, y_train) # hypothesis와 레이블로 cost 정의

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609258
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269521
Epoch 1000/1000 Cost: 0.241922
